In [70]:
%%file archivo.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

void genRamdon(int LEN, int MAX, float *v);
void cPromedio(int LEN, float *v, float *cprom);
extern void asmPromedio(int LEN, float *v, float *asmprom);
//  Calling conv 64 bits -> rdi       rsi       rdx

int main(){
    
    srand(time(NULL));
    
    /*Para medir los tiempos*/
    clock_t start, stop;
    
    float *v;
    float asmprom, cprom;
    int LEN = 4096*2, MAX = 100;
    
    v = (float *)malloc(LEN*sizeof(float));
    
    genRamdon(LEN, MAX, v);
    start = clock();
    asmPromedio(LEN, v, &asmprom);
    stop = clock();
    printf("El promedio en ASM es: %.4f\n", asmprom);
    printf("El tiempo de asmPromedio es: %.4f us\n", 1e6*(float)(stop-start)/CLOCKS_PER_SEC);
    // SALE EN MILLON DE SEGUNDOS, por eso tienes que convertir a segundos
    // CLOCKS_PER_SEC <- 1 millon
    
    start = clock();
    cPromedio(LEN, v, &cprom);
    stop = clock();
    printf("El promedio en C es: %.4f\n", cprom);
    printf("El tiempo de cPromedio es: %.4f us\n", 1e6*(float)(stop-start)/CLOCKS_PER_SEC);
    
    return 0;
}


void genRamdon(int LEN, int MAX, float *v){
    /*Funcion para generar numeros aleatorios*/
    for(int i = 0; i < LEN; i++){
        v[i] = rand() % MAX;
    }
}

void cPromedio(int LEN, float *v, float *cprom){
    float sum = 0.0;
    for (int i = 0; i < LEN; i++){
        sum += v[i];
    }
    (*cprom) = sum/(float)LEN;
}

Overwriting archivo.c


In [71]:
%%file asmPromedio.asm

global asmPromedio
    section .text
    
asmPromedio:
    ; Prologo
    push rbp
    mov rbp, rsp
    push rsi
    push rdi
    push rbx
        
    ; Calling conv 64 bits -> LEN: rdi, *v: rsi, *asmprom: rdx
    
    xorps xmm0, xmm0
    xorps xmm1, xmm1             ; sum = 0.0
    xorps xmm2, xmm2
    
    cvtsi2ss xmm2, rdi           ; LEN
    
    cmp rdi, 0                   ; Si no tenemos datos slaimos
    je fin
    
bucle:
    movss xmm0, [rsi]              ; Direccion primer elemento
    addss xmm1, xmm0
    add rsi, 4
    dec rdi                        ; Cuando este llegue a cero se levantara la bandera de cero
    jnz bucle
    
    
    divss xmm1, xmm2             ; sum/LEN
    
fin:
    movss [rdx], xmm1            ; 
    
    ; Epilogo
    pop rdx
    pop rdi
    pop rsi
    mov rsp, rbp
    pop rbp
    
    ret
    

Overwriting asmPromedio.asm


Flotante <- para el exponente tiene menos bits que el double
double <- exponente de un double son 11 bits
float <- exponente de un float son 8 bits

SIMD <- Empaquetar

## Probando los codigos

In [66]:
! nasm -f elf64 asmPromedio.asm -o asmPromedio.o
! gcc archivo.c asmPromedio.o -o promedio

In [67]:
! ./promedio

El promedio en ASM es: 58.2500
El tiempo de asmPromedio es: 1.0000 us
El promedio en C es: 58.2500
El tiempo de cPromedio es: 0.0000 us


OPtimizaciones <- Por eso parece que en c es más rapido

 -O0            Para que c no realice ninguna operacion

In [72]:
! nasm -f elf64 asmPromedio.asm -o asmPromedio.o
! gcc -O0 archivo.c asmPromedio.o -o promedio

In [73]:
! ./promedio

El promedio en ASM es: 49.5613
El tiempo de asmPromedio es: 7.0000 us
El promedio en C es: 49.5613
El tiempo de cPromedio es: 21.0000 us


In [ ]:
! rm .c
! rm .asm